# Campaign Email Opens
This API integration stores the data of campaigns in which the recipients opened the mails


In [3]:
import requests
from base64 import b64encode
import json
import pandas as pd
from datetime import datetime, date, timedelta

import sys
import yaml as yml
import pyodbc as cnn

import re
from dateutil.relativedelta import relativedelta
from tqdm import tqdm, trange
   
apikey = "fz/ocDZn9rHm9ZAupGsDNjoBZGSnHO+8vmU/xWiPPvvFzPE0STJnTbnvu6/FvyOpUhp/TBu0txZ4xZYz6J8iyxkUDkBowVTiDflAvH0qdi5NzWSW+iq8L7j77/Qg77/6Wkj1JvxGkBTOtFOpV9saGw=="
clientId = "33604187fc197c87eb8d2f553935ef8e"

page = 1
pagesize = 1000

# Encode the credentials with base64
username = apikey
password = ''
encoded_credentials = b64encode(bytes(f'{username}:{password}',encoding='ascii')).decode('ascii')

# Use Basic Authorization
auth_header = f'Basic {encoded_credentials}'
print(f'Auth header: {auth_header}')

Auth header: Basic Znovb2NEWm45ckhtOVpBdXBHc0ROam9CWkdTbkhPKzh2bVUveFdpUFB2dkZ6UEUwU1RKblRibnZ1Ni9GdnlPcFVocC9UQnUwdHhaNHhaWXo2SjhpeXhrVURrQm93VlRpRGZsQXZIMHFkaTVOeldTVytpcThMN2o3Ny9RZzc3LzZXa2oxSnZ4R2tCVE90Rk9wVjlzYUd3PT06


In [4]:
#### Open the config file
with open(r'dbconfig.yaml') as file:
    config = yml.full_load(file)

In [5]:
########## Get the Sent Campaigns #########
# Get the Campaign Id from this API for future campaign - related API
campaign_detail_url = f'https://api.createsend.com/api/v3.2/clients/{clientId}/campaigns.json'
response = requests.get(campaign_detail_url, headers={'Authorization': auth_header } ) 
response_campaign_list = response.json()
df_campaign_list = pd.DataFrame.from_dict(response_campaign_list)
df_campaign_list.head()

,Name,FromName,FromEmail,ReplyTo,SentDate,TotalRecipients,CampaignID,Subject,WebVersionURL,WebVersionTextURL
0,Tin's School Fund Campaign,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-08 16:30:00,5,8fd3ba6bd1dd1d58ce10cd28a12f3795,Tin's School Fund Campaign,http://createsend.com/t/t-2792275A54D1EB4C2540...,http://createsend.com/t/t-2792275A54D1EB4C2540...
1,Tin's Test Campaign for Breakfast,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-07 14:20:00,3,319f7bddfdce8c093e96c39726e62048,Tin's Test Campaign,http://createsend.com/t/t-8398EA282987899E2540...,http://createsend.com/t/t-8398EA282987899E2540...
2,Breakfast for Students Campaign,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-01 17:37:00,1,7fac9427e46cd56cf13fed5c723d7801,Tin's Campaign,http://createsend.com/t/t-B4D1C2432919123A2540...,http://createsend.com/t/t-B4D1C2432919123A2540...


# Filter out the Sent Campaigns older than 4 months

In [6]:
def get_datetime_Ndays_ago(n):
    N_days_ago_datetime = datetime.now() - timedelta(days=n)
    return N_days_ago_datetime.strftime("%Y-%m-%d %H:%M")

In [9]:
### Filter date should come from config file, 120 days equivalent to 4 months
active_campaign_day = 120
active_period = get_datetime_Ndays_ago (active_campaign_day)

all_active_campaign = df_campaign_list['SentDate'] > active_period 
campaigns_on_active_period = df_campaign_list.loc[all_active_campaign]

campaigns_on_active_period

,Name,FromName,FromEmail,ReplyTo,SentDate,TotalRecipients,CampaignID,Subject,WebVersionURL,WebVersionTextURL
0,Tin's School Fund Campaign,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-08 16:30:00,5,8fd3ba6bd1dd1d58ce10cd28a12f3795,Tin's School Fund Campaign,http://createsend.com/t/t-2792275A54D1EB4C2540...,http://createsend.com/t/t-2792275A54D1EB4C2540...
1,Tin's Test Campaign for Breakfast,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-07 14:20:00,3,319f7bddfdce8c093e96c39726e62048,Tin's Test Campaign,http://createsend.com/t/t-8398EA282987899E2540...,http://createsend.com/t/t-8398EA282987899E2540...
2,Breakfast for Students Campaign,Tin,hninpwint.tin@student.uts.edu.au,hninpwint.tin@student.uts.edu.au,2020-12-01 17:37:00,1,7fac9427e46cd56cf13fed5c723d7801,Tin's Campaign,http://createsend.com/t/t-B4D1C2432919123A2540...,http://createsend.com/t/t-B4D1C2432919123A2540...


### API Requests in Iteration

In [12]:
campaingn_opens_dataset = pd.DataFrame()
all_records = []
n_past_days = 120
params = {"page": page, "pagesize": pagesize, "orderfield": "email", "orderdirection": "asc" }
#desired_columns = ["campaignId", "EmailAddress", "Date", "CountryCode", "ListID"]
desired_columns = ["campaignId", "EmailAddress", "Date"]

for campaign_id in campaigns_on_active_period["CampaignID"]:

    url_campaign_opens = f"https://api.createsend.com/api/v3.2/campaigns/{campaign_id}/opens.json"
    df_campaign_email_opens = pd.DataFrame()
    for x in range(n_past_days):
        
         N_days_ago_datetime = get_datetime_Ndays_ago(x)
         # Pass N days if not full load
         params["date"] = N_days_ago_datetime
         response = requests.get(url_campaign_opens, headers={'Authorization': auth_header }, params = params) 
         #print(response.json())
         response_campaign_opens = response.json()
         if(len(response_campaign_opens["Results"]) == 0): continue
         #print("Data exists")
         response_dataframe = pd.DataFrame.from_dict(response_campaign_opens["Results"])
         response_dataframe["campaignId"] = campaign_id
         
         df_campaign_email_opens = df_campaign_email_opens.append(response_dataframe, ignore_index=True, sort=False) 
         
    
    all_records.append(df_campaign_email_opens)
    

#print("All record length", len(all_records))
if(len(all_records) > 0):
    campaingn_opens_dataset = pd.concat(all_records, axis=0, ignore_index=True, sort=False) 
    campaingn_opens_dataset = campaingn_opens_dataset[desired_columns]
    campaingn_opens_dataset = campaingn_opens_dataset.drop_duplicates()


print("Total Number of Records", campaingn_opens_dataset.shape)

TypeError: concat() got an unexpected keyword argument 'delim_whitespace'

In [11]:
## Print The final dataframe
#print("Total Number of Records", campaingn_clicks_dataset.shape)
campaingn_opens_dataset

,campaignId,EmailAddress,Date
0,8fd3ba6bd1dd1d58ce10cd28a12f3795,ashish.kulkarni@live.com,2020-12-10 11:53:00
2,8fd3ba6bd1dd1d58ce10cd28a12f3795,ashish.kulkarni@live.com,2020-12-08 22:45:00
5,8fd3ba6bd1dd1d58ce10cd28a12f3795,ashish@phillip.com.sg,2020-12-08 16:53:00
7,8fd3ba6bd1dd1d58ce10cd28a12f3795,ashish@phillip.com.sg,2020-12-08 16:55:00
9,8fd3ba6bd1dd1d58ce10cd28a12f3795,hninpt@gmail.com,2020-12-08 16:33:00
776,319f7bddfdce8c093e96c39726e62048,hninpt@gmail.com,2020-12-08 14:58:00
777,319f7bddfdce8c093e96c39726e62048,hninpt@gmail.com,2020-12-08 15:01:00
778,319f7bddfdce8c093e96c39726e62048,hninpt@gmail.com,2020-12-08 10:58:00
1033,7fac9427e46cd56cf13fed5c723d7801,deepuinuk@gmail.com,2020-12-01 17:40:00
1034,7fac9427e46cd56cf13fed5c723d7801,deepuinuk@gmail.com,2020-12-02 08:11:00


### Establish DB Connection

In [50]:
def insert_to_db(row, cursor):
    cursor.execute('  INSERT  INTO '
                   '          campaign_email_opens '
                   '          (campaign_id, email_address, date_opened )'
                   '  VALUES   (?,?,?)',
                   row['campaignId'], row['EmailAddress'], row['Date'])
    
# Open the config file
with open(r'dbconfig.yaml') as file:
    config = yml.full_load(file)

# Establish the connection with database server
conn = cnn.connect('DRIVER='+config['instances']['driver']+
                   ';SERVER='+config['instances']['servername']+
                   ';DATABASE='+config['instances']['database']+
                   ';UID='+config['instances']['username']+
                   ';PWD='+config['instances']['password'])

conn.autocommit = False

#check loading configuration
#METHOD: full/incremental
cfg_load_method = config['loading']['method']

### Insert  API Data into Database, Table -  'campaign_email_opens'

In [53]:

### This should be UPDATE #####
cursor = conn.cursor()

try:
    #if full-load, delete all the records from the table
    if (cfg_load_method == 'full'):
        cursor.execute('DELETE FROM campaign_email_opens')
    
    #insert dataframe to DB
    campaingn_clicks_dataset.apply(insert_to_db, cursor = cursor, axis = 1)
      
    conn.commit()
except:
    conn.rollback()
    print(sys.exc_info())
finally:  
    cursor.close()

### Testing the result

In [49]:
#### Testing the result #####
result.groupby("campaignId")['EmailAddress'].count().reset_index()
   
### Print the number of unique values in each columns
result.groupby('campaignId').nunique()

,campaignId,EmailAddress,Date
campaignId,,,
319f7bddfdce8c093e96c39726e62048,1,1,3
7fac9427e46cd56cf13fed5c723d7801,1,1,2
8fd3ba6bd1dd1d58ce10cd28a12f3795,1,3,5
